In [1]:
#Imports
import os
import random
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, MWETokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from functools import reduce
from collections import Counter
import spacy
from spacy.matcher import Matcher
from spacy import displacy
import visualise_spacy_tree
from IPython.display import Image, display
from nltk.stem import PorterStemmer
from nltk.corpus import conll2000
from gtts import gTTS
import IPython
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('conll2000')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jackd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jackd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jackd\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\jackd\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\conll2000.zip.


True

In [2]:
#Read the data into a dataframe
file = pd.read_csv(r'./data_assessment_2/data_for_info_retriev_extract/Dataset_IMDB.csv')
df = pd.DataFrame(file)
df

,Id,Text,Author,Rating
0,29420,"In my opinion, a movie reviewer's most importa...",Schwartz,0.1
1,17219,STARSHIP TROOPERS (director: Paul Verhoeven; c...,Schwartz,0.2
2,18406,"THE SCHOOL OF FLESH (ECOLE DE lA CHAIR, L') (D...",Schwartz,0.2
3,18648,"LOCK, STOCK AND TWO SMOKING BARRELS (director:...",Schwartz,0.2
4,20021,RUN LOLA RUN (LOLA RENNT)(director/writer: Tom...,Schwartz,0.2
...,...,...,...,...
5001,7470,The conventional wisdom is that movie sequels ...,Rhodes,0.9
5002,7853,Nicolas Roeg's mesmerizing 1971 film WALKABOUT...,Rhodes,0.9
5003,8309,The movie AIR FORCE ONE should require a docto...,Rhodes,0.9
5004,8912,"""Well, Jones, at least you haven't forgotten h...",Rhodes,0.9


In [3]:
#First we must 

In [4]:
def token_w(text):#tokenize the data
    text = re.sub(re.compile(r'[^a-zA-Z0-9/s]'),' ',text)
    text = re.sub('[0-9]','',str(text))
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    return words

def inverted_index(words):#create a inverted index
    inverted = {}
    for index, word in enumerate(words):
        locations = inverted.setdefault(word,[])
        locations.append(index)
    return inverted

def inverted_index_add(inverted, doc_id, doc_index):
    for word in doc_index.keys():
        locations = doc_index[word]
        indices = inverted.setdefault(word,{})
        indices[doc_id] = locations
    return inverted

In [5]:
inverted_data_index = {}
data_index = []
tokens_store = {}
stop_words = nltk.corpus.stopwords.words('english')
review_id = 0

#for each row in the df create a inverted index for later use
for index, row in tqdm(df.iterrows()):
    words = token_w(row['Text'])
    tokens_store[review_id] = words
    review_index = inverted_index(words)
    inverted_index_add(inverted_data_index, review_id, review_index)
    data_index.append(row['Id'])
    review_id = review_id +1

5006it [00:13, 369.35it/s]


In [6]:
#This process only finds the indexes where one of our words is met, to get an accurate output
#For the movie we our looking for we must employee a boolean search to connect the words together

In [7]:
def boolean_search(inverted, idexes, search):#search for the terms given
    words = [word for _, word in enumerate(token_w(search)) if word in inverted]
    results = [set(inverted[word].keys()) for word in tqdm(words)]
    docs = reduce(lambda x, y: x & y, results) if results else []
    return ([idexes[doc] for doc in docs])

#print("Search using AND:", boolean_search(inverted_data_index,data_index,starwars,'AND'))
#print("Search using OR:", boolean_search(inverted_data_index,data_index,starwars,'OR'))

#Checks for pressence of both the terms given eg must contain loin and king not just one

In [8]:
def clean(text):
    #Remove numbers
    text = re.sub('[0-9]+','',str(text))
    #remove new lines
    text = re.sub('/n ','',str(text))
    text = re.sub('/n',' ',str(text))
    text = re.sub('/t','',str(text))
    #remove apostrophes
    text = re.sub("'s",'',str(text))
    #remove hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("- ",'',str(text))
    #remove quotes
    text = re.sub('\"','',str(text))
    #Remove mr and mrs
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    return text

In [9]:
df['Clean_Text'] = df['Text'].apply(clean)
df.head()

,Id,Text,Author,Rating,Clean_Text
0,29420,"In my opinion, a movie reviewer's most importa...",Schwartz,0.1,"In my opinion, a movie reviewer most important..."
1,17219,STARSHIP TROOPERS (director: Paul Verhoeven; c...,Schwartz,0.2,STARSHIP TROOPERS (director: Paul Verhoeven; c...
2,18406,"THE SCHOOL OF FLESH (ECOLE DE lA CHAIR, L') (D...",Schwartz,0.2,"THE SCHOOL OF FLESH (ECOLE DE lA CHAIR, L') (D..."
3,18648,"LOCK, STOCK AND TWO SMOKING BARRELS (director:...",Schwartz,0.2,"LOCK, STOCK AND TWO SMOKING BARRELS (director:..."
4,20021,RUN LOLA RUN (LOLA RENNT)(director/writer: Tom...,Schwartz,0.2,RUN LOLA RUN (LOLA RENNT)(director/writer: Tom...


In [10]:
def sentences(text):#split the data into seperate sentences
    text = re.split('[.]',text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent

df['sent'] = df['Clean_Text'].apply(sentences)#apply sentece function and append results to new df column

df2 = pd.DataFrame(columns=['Id','sent','rating'])
#create a new df with sentence defined
row_list = []

for i in tqdm(range(len(df))):
    for sent in df.loc[i,'sent']:
        ID = df.loc[i,'Id']
        ratings = df.loc[i,'Rating']
        dict1 = {'Id':ID,'sent':sent,'Rating':ratings}
        row_list.append(dict1)
        
df2 = pd.DataFrame(row_list)
df2

100%|████████████████████████████████████████████████████████████████████████████| 5006/5006 [00:01<00:00, 2958.26it/s]


,Id,sent,Rating
0,29420,"In my opinion, a movie reviewer most important...",0.1
1,29420,"But with Soul Survivors, I'm so full of criti...",0.1
2,29420,"\nHowever, protocol forces me to do otherwise",0.1
3,29420,Soul Survivors tells the story of four colleg...,0.1
4,29420,Party girl Annabel (Eliza Dushku of Bring It ...,0.1
...,...,...,...
191281,9085,* = Poor show,0.9
191282,9085,Don't waste your money,0.9
191283,9085,= Totally and painfully unbearable picture,0.9
191284,9085,"\nREVIEW WRITTEN ON: September , \nOpinions ex...",0.9


In [11]:
#Helper function to find director

from collections import Counter

def Most_Common(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]

Run all cells previous to this one

The next cell requires a movie inout and will fetch the required results

In [24]:
dfComplte = pd.DataFrame(columns=['Movie_Name','Director','Reviews_About_Movie', 'Reviews_Mention_Movie','Rating'])
#new_df for results

lang_model = spacy.load("en_core_web_sm")
def find_movie(text):
    t = a.split()#splits users input into words
    names = []
    doc = lang_model(text)
    
    #finds sentences that contain the disred movie
    matcher = Matcher(lang_model.vocab)
    matcher.add("PERSON", [[{"lower": t[0]}, {"lower": t[1]}]])
    matches = matcher(doc)
    
    for i in range(0,len(matches)):
        token = doc[matches[i][1]:matches[i][2]]
        names.append(str(token))
    return names
#--------------------------find movie asked for ------------------------------
a = input()

movie = (boolean_search(inverted_data_index,data_index,a))#searchs for movie

df_movie = pd.DataFrame(columns=['Id','sent','rating'])
temp = []

#create df only contain reviews about the movie searched for
for i in tqdm(range(0,len(movie))):
    for j in range(len(df2)):
        if df2['Id'][j] == movie[i] :
            ID2 = df2.loc[j,'Id']
            sent2 = df2.loc[j,'sent']
            ratings2 = df2.loc[j,'Rating']
            dict2 = {'Id':ID2,'sent':sent2,'Rating':ratings2}
            temp.append(dict2)
            
df_movie = pd.DataFrame(temp)
#-------------------------------------------------------------------------------------------
#----------------------------------find movies that mention the title--------------------------
df_movie['Names'] = df_movie['sent'].apply(find_movie)
#-------------------------------------------------------------------------------------
#--------------------------Find reviews about movie-----------------------------------
l = []
temp = {}
y = a.upper()
tem = y.split()
for i in tqdm(range(len(df_movie))):
    if len(df_movie.loc[i,'Names'])!=0:
        doc = lang_model(df_movie.loc[i,'sent'])
        i = df_movie.loc[i, 'Id']
        for token in doc:
            if(token.dep_=='nsubj'):
                if token.text == tem[1]:
                    temp[i] = token
                if token.text == tem[0]:
                    temp[i] = token
#--------------------------------------------------------------------------------------
#-------------------------------outputting-------------------------------------
temp_list =[]
movie_review_id = []
review_not = []
temp_rating = []
director = []
for key, value in temp.items():
    movie_review_id.append(key)
    
for i in range(len(movie)):
    if movie[i] not in movie_review_id:
        review_not.append(movie[i])
        
for j in movie_review_id:
    for i in range(len(df)):
        if j == df.loc[i,'Id']:
            temp_rating.append(df.loc[i,'Rating'])
            
for i in movie_review_id:
    for j in range(len(df2)):
        if df2.loc[j,'Id'] == i:
            words = token_w(df2.loc[j,'sent'])
            for n, word in enumerate(words):
                if 'director' in word:
                    director.append(' '.join(words[n+1:n+3]))

new_d = Most_Common(director)
rating = sum(temp_rating)/len(temp_rating)
        
dict1 = {'Movie_Name': a,'Director':new_d,'Reviews_About_Movie':movie_review_id, 'Reviews_Mention_Movie':review_not,'Rating':rating}
temp_list.append(dict1)
dfComplte = pd.DataFrame(temp_list)
dfComplte.head()

star wars


100%|████████████████████████████████████████████████████████████████████████████| 4094/4094 [00:00<00:00, 4433.31it/s]


,Movie_Name,Director,Reviews_About_Movie,Reviews_Mention_Movie,Rating
0,star wars,george lucas,"[18433, 7309, 6829, 6979, 6782, 6949, 18265, 7...","[4550, 6348, 2035, 18767, 7163, 7015, 8245, 12...",0.673077


In [25]:
print(movie_review_id)
print(review_not)

[18433, 7309, 6829, 6979, 6782, 6949, 18265, 7114, 5049, 5055, 24551, 24563, 25020]
[4550, 6348, 2035, 18767, 7163, 7015, 8245, 12602, 10897, 18485, 6383, 4125, 12547, 2985, 20096, 7096, 7624, 11641, 2582, 27812, 28755, 11025, 28772, 28787, 5616, 4634, 7676, 12057, 4486, 26197, 5100, 7065, 26895, 16619, 18779, 3040, 6971, 7536, 24553, 6417, 3785, 22991, 7197, 7533, 29716, 2483, 25302, 5574, 9638, 12890, 10230, 25766, 5592, 15031, 11468, 7212, 17316, 13536, 3023, 13636, 14953, 21544, 15420, 22584, 29024, 24561, 21610, 3852, 28970, 5542, 20185, 3742]


In [18]:
#determine of the movie is a good reccomendation
if rating > 0.5:
    gorb = 'good'
else:
    gorb = 'bad'
    
#Text to be read
text = """You have requested information about {Movie} Movie, 
the director of this movie is {Director}, there is a total of {Reviews} reviews about this movie,
There is {Mentions} Mentions of this movie in other reviews
The average score of this movie is {Rating} making it a {recomend} movie to watch.
""".format(Movie=a, Director=new_d, Reviews=len(movie_review_id), Mentions=len(review_not), Rating=rating, recomend=gorb)

filename = "Task3.mp3"#Text saved to file
tts = gTTS(text, lang="en", tld='co.uk', slow=False)
tts.save(filename)
IPython.display.Audio(filename,autoplay=True)#Read text above

In [15]:
text

'You have requested information about starship troopers Movie, \nthe director of this movie is paul verhoeven, there is a total of 4 reviews about this movie,\nThere is 14 Mentions of this movie in other reviews\nThe average score of this movie is 0.625 making it a good movie to watch.\n'